In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

NormalBeats_LeadI = np.load('/content/drive/MyDrive/PUCP/2023-1/INTRO A SEÑALES/Proyecto Final/array_Normal_beats_leadI.npy')
LBBBBeats_LeadI = np.load('/content/drive/MyDrive/PUCP/2023-1/INTRO A SEÑALES/Proyecto Final/array_LBBB_beats_leadI.npy')
AtrialFibBeats_LeadI = np.load('/content/drive/MyDrive/PUCP/2023-1/INTRO A SEÑALES/Proyecto Final/array_AtrialFib_beats_leadI.npy')
LBBBBeats_LeadI_test = np.load('/content/drive/MyDrive/PUCP/2023-1/INTRO A SEÑALES/Proyecto Final/array_LBBB_beats_leadI_test.npy')

Feature Extraction

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import skew, kurtosis
import pywt
from scipy.stats import skew, kurtosis, median_abs_deviation
#Defino funcion para la extraccion de caracteristicas
def extract_features(data):
    features = []
    for beat in data:
        """ WAVELET TRANSFORM """ # EXTRACCION DE CARACTERISTICAS (WAVELET DECOMPOSITION): db4, nivel 1 (minimo nivel y el que me da la mejor caracterizacion de un beat de ecg)
        wav_coef = pywt.wavedec(beat, wavelet, level = level); #retorno de coeficiente de aproximacion (approx coeff son los primeros) y coeficientes de detalle (detail coeff son los segundos)
        features.append([np.sqrt(np.mean(np.square(wav_coef[0]))), np.linalg.norm(wav_coef[0],ord=2), np.linalg.norm(wav_coef[0],ord=1), np.max(wav_coef[0]), np.min(wav_coef[0]), np.mean(wav_coef[0]), np.var(wav_coef[0]), np.std(wav_coef[0]), skew(wav_coef[0]), kurtosis(wav_coef[0]), np.ptp(wav_coef[0]), np.cov(wav_coef[0]), np.corrcoef(wav_coef[0]), np.quantile(wav_coef[0], 0.5), np.percentile(wav_coef[0], 50), median_abs_deviation(wav_coef[0]), np.sqrt(np.mean(np.square(wav_coef[1]))), np.linalg.norm(wav_coef[1],ord=2), np.linalg.norm(wav_coef[1],ord=1), np.max(wav_coef[1]), np.min(wav_coef[1]), np.mean(wav_coef[1]), np.var(wav_coef[1]), np.std(wav_coef[1]), skew(wav_coef[1]), kurtosis(wav_coef[1]), np.ptp(wav_coef[1]), np.cov(wav_coef[1]), np.corrcoef(wav_coef[1]), np.quantile(wav_coef[1], 0.5), np.percentile(wav_coef[1], 50), median_abs_deviation(wav_coef[1])])

    return np.array(features)

wavelet = 'db4'
level = 1

# Extraccion de caracteristicas de cada tipo de beat
NormalBeats_LeadI_features = extract_features(NormalBeats_LeadI)
LBBBBeats_LeadI_features = extract_features(LBBBBeats_LeadI)
AtrialFibBeats_LeadI_features = extract_features(AtrialFibBeats_LeadI)

Random Forest Algorithm

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


''' LABELS '''
NormalBeats_LeadI_labels = np.zeros(len(NormalBeats_LeadI_features))
LBBBBeats_LeadI_labels = np.ones(len(LBBBBeats_LeadI_features))
AtrialFibBeats_LeadI_labels = np.full(len(AtrialFibBeats_LeadI_features), 2)

# Concatenate the features and labels
features = np.concatenate((NormalBeats_LeadI_features, LBBBBeats_LeadI_features, AtrialFibBeats_LeadI_features), axis=0)
labels = np.concatenate((NormalBeats_LeadI_labels, LBBBBeats_LeadI_labels, AtrialFibBeats_LeadI_labels), axis=0)


''' SPLITING THE DATA '''
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

''' Training '''
rf = RandomForestClassifier(n_estimators=1000, random_state=42)

# Training the classifier
rf.fit(X_train, y_train)

# Making predictions with the test set
y_pred = rf.predict(X_test)

# Accuracy
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2%}".format(accuracy))

precision_per_label = precision_score(y_test, y_pred, average=None)
precision_av = precision_score(y_test, y_pred, average="weighted")
print("Precision per class: {}".format(["{:.2%}".format(p) for p in precision_per_label]))
print("Average Precision: {:.2%}".format(precision_av))

recall_per_label = recall_score(y_test, y_pred, average=None)  # Sensitivity
recall_av = recall_score(y_test, y_pred, average="weighted")  # Sensitivity
print("Recall per class: {}".format(["{:.2%}".format(r) for r in recall_per_label]))
print("Average Recall: {:.2%}".format(recall_av))

f1_per_label = f1_score(y_test, y_pred, average=None)
f1_av = f1_score(y_test, y_pred, average="weighted")
print("F1 Score per class: {}".format(["{:.2%}".format(f) for f in f1_per_label]))
print("Average F1 Score: {:.2%}".format(f1_av))

Accuracy: 89.14%
Precision per class: ['85.11%', '95.93%', '87.03%']
Average Precision: 89.06%
Recall per class: ['84.15%', '98.33%', '86.15%']
Average Recall: 89.14%
F1 Score per class: ['84.63%', '97.12%', '86.59%']
Average F1 Score: 89.10%


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

''' LABELS '''
NormalBeats_LeadI_labels = np.zeros(len(NormalBeats_LeadI_features))
LBBBBeats_LeadI_labels = np.ones(len(LBBBBeats_LeadI_features))
AtrialFibBeats_LeadI_labels = np.full(len(AtrialFibBeats_LeadI_features), 2)

# Concatenate the features and labels
features = np.concatenate((NormalBeats_LeadI_features, LBBBBeats_LeadI_features, AtrialFibBeats_LeadI_features), axis=0)
labels = np.concatenate((NormalBeats_LeadI_labels, LBBBBeats_LeadI_labels, AtrialFibBeats_LeadI_labels), axis=0)


''' SPLITING THE DATA '''
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

param_grid = {
    'n_estimators': [500, 1000, 1500],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

In [ ]:
for test_beat in LBBBBeats_LeadI_test:
    # Extract features for the test beat
    test_features = extract_features([test_beat])

    # Make predictions on the test beat using the SVM classifier
    prediction = rf.predict(test_features)

    # Determine the label based on the prediction
    if prediction == 0:
        label = "Normal"
    elif prediction == 1:
        label = "LBBB"
    else:
        label = "AtrialFib"

    print("Predicted label:", label)

Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted 

In [ ]:
type(NormalBeats_LeadI_features)

numpy.ndarray